# What app should i create next?

**Goals:** 
1. Decide what properties are valued better on average amogst mobile app users.
2. What platform is better to for release.

In this project i'm going to evaluate different app attributes to decide which characteristics are better perceived by the market. Attributes that's going to be evaluated are app language, content rating, category, is app paid or no. Evaluation parameters are app's average number of reactions (count of rating scores assigned) and / or number installs, and average rating. High number of reactions tells us about the demand for the application and high potential income. High average rating indicates that the audience is more loyal to such type of applications.<br/>The final result will be a full set of application characteristics, determined by indirect evidence. 

To achieve second goal we are going to answer two questions: "Are the same paid apps on ios have more users?" and "Are there any apps that are free on android and paid on ios?". An affirmative answer to both questions will make the Appstore more attractive for releasing a new application.

I'm going to use two datasets provided by kaggle users, one for each platform. 
* [Google Store dataset](https://www.kaggle.com/gauthamp10/google-playstore-apps) includes Google Play Store App data of 600K+ applications.
* [Apple Store dateset](https://www.kaggle.com/cmqub19/763k-ios-app-info) includes data on 760K+ applications.

*Real goal of the project to practice pandas exploratory analysis and visualization with seaborn module*

### Conclusions:
* Our main idea is to make free english puzzle/word game for Play Market and second best option is eglish paid english photo/video app for Appstore
* Free applications find a better response in Play Market, while the number and ratio of paid applications is more attractive in Appstore.

## Preparing data

In [ ]:
# Importing required modules
import os
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# To get file paths with connected datasets in Kaggle we are going to run:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Reading datasets into pandas DataFrame objects
df_google = pd.read_csv('/kaggle/input/google-playstore-apps/Google-Playstore.csv', low_memory=False)
df_apple = pd.read_csv('/kaggle/input/763k-ios-app-info/763K_plus_IOS_Apps_Info.csv', low_memory=False)

In [ ]:
# Isolating columns we need
df_ggl = df_google[['App Name', 'Category', 'Rating', 'Rating Count', 'Maximum Installs', 'Price',
                    'Content Rating', 'Ad Supported', 'In App Purchases', 'Editors Choice']]
df_ggl.columns = ['app_name', 'category', 'rating', 'rating_count', 'max_installs', 'price', 'content_rating', 'ads', 'in_apps', 'editors']

df_ios = df_apple[['Title', 'Age_Rating', 'Total_Average_Rating', 'Total_Number_of_Ratings', 'Price_USD', 'Primary_Genre', 'Languages']]
df_ios.columns = ['app_name', 'content_rating', 'rating', 'rating_count', 'price', 'category', 'language']
df_ios = df_ios[['app_name', 'category', 'rating', 'rating_count', 'price', 'content_rating', 'language']]

In [ ]:
# Inspect new dataframe
print(df_ggl.info())
df_ggl.head(5)

This version of Google dataset is cleaned and prepared for analysis, but Apple dataset needs some cleaning, so we are going to start with it

In [ ]:
# Removing duplicated data
print('Duplicates:\n',df_ggl.duplicated().value_counts())

In [ ]:
# Null values are met only in rating columns that means these apps most likely are new. Let's drop them for n
df_ggl = df_ggl.dropna()

## Cleaning ios dataframe
1. Look for duplicates.
2. Check null-values.
3. Negative ratings - what's that?
4. Categorize content rating in both dataframes.

In [ ]:
# Inspect new dataframe
print(df_ios.info())
df_ios.head()

In [ ]:
# Removing duplicated data
print('Before:',df_ios.duplicated().value_counts())
df_ios.drop_duplicates(inplace=True)
print('After:',df_ios.duplicated().value_counts())

In [ ]:
# Checking null-values (1 null value - drop with ease)
print('Nulls:', df_ios.isna().value_counts())
df_ios.dropna(inplace=True)
print('No nulls:', df_ios.isna().value_counts())

In [ ]:
# Checking negative values in rating column
df_ios[df_ios['rating'] < 0]['rating_count'].value_counts()

Big part of given dataset doesn't have rating and rating_count information. We'll continue with the remaining part and cut this part of dataset as it's not clear for me how these values were obtained. 

I suggest cutting off a portion of our data that includes applications with low response rates as we are looking for most popular applications. 
This will also help us to reduce the impact of new and niche applications on the average rating considered within attribute scores.

In [ ]:
# Getting rid of null/negative rating and rating_counts
print('Apps in the Ios dataset before:', df_ios.shape[0])
print('Apps in the Google dataset before:', df_ggl.shape[0])
print()
df_ios = df_ios[df_ios['rating'] > 0]
df_ggl = df_ggl[df_ggl['rating'] > 0]
print('Apps left in the Ios dataset after:', df_ios.shape[0])
print('Apps left in the Google dataset after:', df_ggl.shape[0])

In [ ]:
# Also let's cut apps with low rating_count - not very popular (yet) apps
print('Apps in the Ios dataset before:', df_ios.shape[0], 'and % of low rating_count apps:', 
      df_ios['rating_count'].value_counts()[:100].sum() / df_ios['rating_count'].shape[0] * 100 )
print('Apps in the Google dataset before:', df_ggl.shape[0], 'and % of low rating_count apps:', 
      df_ggl['rating_count'].value_counts()[:100].sum() / df_ggl['rating_count'].shape[0] * 100 )
df_ios = df_ios[df_ios['rating_count'] >= 100]
df_ggl = df_ggl[df_ggl['rating_count'] >= 100]
print('Apps left in the Ios dataset after:', df_ios.shape[0])
print('Apps left in the Google dataset after:', df_ggl.shape[0])

### IOS rating columns is categorical data!

We must keep in mind that ratings in ios dataset are categorical as shown below.

In [ ]:
df_ios[df_ios['rating'] > 0]['rating'].value_counts()

### Can we say, that number of installs depends on number of ratings? 

The data on installs is only available in the Play Market dataset, so let's build a model for it

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
# If we want to make predictional models we need to get rid of outliers
df_ggl[(df_ggl['rating_count'] < 50000) & (df_ggl['max_installs'] < 10000000)].shape[0] / df_ggl.shape[0]

In [ ]:
# Drawing dependency plot 
ax = sns.regplot(data=df_ggl[(df_ggl['rating_count'] < 50000) & (df_ggl['max_installs'] < 10000000)], x='rating_count', y='max_installs')

In [ ]:
# Fit simple linear regression model
model = ols('max_installs ~ rating_count', data=df_ggl[(df_ggl['rating_count'] < 50000) & (df_ggl['max_installs'] < 10000000)])
model_fit = model.fit()

# View model summary
model_fit.summary()

With this model, we will be able to estimate the number of installations for the required parameters, if necessary. We can also predict the demand for our application. 

## Comparing overall means of datasets

The main metric for us will be the reactions to app ratio (rating counts mean). The secondary metric will be the average rating. 


In [ ]:
# Drawing comparison table
print(f"{'Platform':>20} \ {'rating':>10} \ {'reactions per app': >30} \ {'totall apps': >20} \ ")
print('-' * 91)
print(f"{'Play Market (Google)':>20} \ {df_ggl['rating'].mean():>10.2f} \ {df_ggl['rating_count'].sum() / df_ggl['app_name'].shape[0]:>30.2f} \ {df_ggl['app_name'].shape[0]:>20} \ ")
print(f"{'Appstore (Apple)':>20} \ {df_ios['rating'].mean():>10.2f} \ {df_ios['rating_count'].sum() / df_ios['app_name'].shape[0]:>30.2f} \ {df_ios['app_name'].shape[0]:>20} \ ")

Apps in Appstore on average are rated better, but have less reactions. The logic here is that we prioritize number of installs (number of reactions in our case which is related) as it's the main stat leading to high monetization untill the average rating is above the mean. High app rating can be transformed into number of install but that's not the rule. Also apps with comparable number of reactions will be prioritized based on rating. For apps or attributes with average rating below mean the decider is difference between mean and actuall rating.

## What language to choose: english or local

Ios dataset contains information about language. For Play Market dataset we are assuming that if application name includes high unicode characters or consists of question marks mostly it's written on non-english language (which is not 100% sure so we need to keep this in mind)

In [ ]:
def english(string):
    ret = 0
    for char in string:
        if ord(char) > 127 or char == '?':
            ret += 1
    if ret > 3:
        return False
    else:
        return True
    
print(english('?????? ? ???? ??????'))
print(english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(english('Docs To Go™ Free Office Suite'))
print(english('Instachat 😜'))

In [ ]:
# Language boolean masks
mask_ios = df_ios['language'].map(lambda x: 'EN' in x)
mask_ggl = df_ggl['app_name'].map(english)

# Splitting our dataframes to compare them
ios_english = df_ios[mask_ios]
ios_local = df_ios[~mask_ios]

ggl_english = df_ggl[mask_ggl]
ggl_local = df_ggl[~mask_ggl]

In [ ]:
# Let's check how our fuction performs on ios dataframe, where we have separate language column
print(f"There are {df_ios[(~df_ios['app_name'].map(english)) & (mask_ios)]['app_name'].count()} apps our function defined as non-english when they actually are")
print(f"Out of {df_ios.shape[0]} apps in totall and {ios_local.shape[0]} non-english apps in Appstore dataset")

In [ ]:
plt.style.use('fivethirtyeight')
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(18, 6))

#Printing KDE for each group
graph = sns.kdeplot(ggl_english['rating'], color='#FF961F', label='English apps', bw=.1, ax=ax1)
graph = sns.kdeplot(ggl_local['rating'], color='#219ebc', label='Non-english apps', bw=.1, ax=ax1)
graph = sns.kdeplot(df_ggl['rating'], color='#023047', label='All apps', bw=.1, ax=ax1)

#Printing their means
graph.axvline(x = df_ggl['rating'].mean(), color = '#023047', linewidth = 3, alpha = .7)
graph.axvline(x = ggl_english['rating'].mean(), color = '#fb8500', linewidth = 3, alpha = .7)
graph.axvline(x = ggl_local['rating'].mean(), color = '#219ebc', linewidth = 3, alpha = .7)

#Graphic details
graph.text(x = 2.1, y = 0.9, s = "Play market", fontsize = 20, weight = 'bold', alpha = .75)
graph.text(x = 2.1, y = 0.85, s = 'app rating depending on fee', fontsize = 16, alpha = .85)
graph.tick_params(axis = 'both', which = 'major', labelsize = 16)
graph.axhline(y = 0, color = 'black', linewidth = 4, alpha = .7)
#graph.set_yticklabels(labels = [])
graph.set_xlim(left = 1.9, right = 5)
graph.xaxis.label.set_visible(False)
ax1.legend(loc='center left', bbox_to_anchor=(0.02, 0.60))

#Printing histograms for categorial data
bins = [1.9, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75, 5]
graph = sns.distplot(ios_english['rating'], norm_hist=True, color='#fb8500',
                     label='English apps', bins=bins, ax=ax2, kde=False, hist_kws={"histtype": "bar","alpha": .7})
graph = sns.distplot(ios_local['rating'], norm_hist=True, color='#219ebc',
                     label='Non-english apps', bins=bins, ax=ax2, kde=False, hist_kws={"histtype": "bar", "alpha": .7})
graph = sns.distplot(df_ios['rating'], norm_hist=True, color='#023047',
                     label='All apps', bins=bins, ax=ax2, kde=False, hist_kws={"histtype": "step", "alpha": 1, 'linewidth': 5})

#Printing their means
graph.axvline(x = ios_english['rating'].mean(), color = '#FF961F', linewidth = 4, alpha = 1)
graph.axvline(x = ios_local['rating'].mean(), color = '#219ebc', linewidth = 3, alpha = 1)
graph.axvline(x = df_ios['rating'].mean(), color = '#023047', linewidth = 3, alpha = 1)

#Graphic details
graph.text(x = 2.1, y = 0.90, s = "Appstore", fontsize = 20, weight = 'bold', alpha = .75)
graph.text(x = 2.1, y = 0.85, s = 'app rating depending on fee', fontsize = 16, alpha = .85)
graph.tick_params(axis = 'both', which = 'major', labelsize = 16)
graph.axhline(y = 0, color = 'black', linewidth = 4, alpha = .7)
#graph.set_yticklabels(labels = [])
graph.set_xlim(left = 1.9, right = 5)
#graph.set_ylim(bottom = 0, top = 0.00001)
graph.xaxis.label.set_visible(False)
ax2.legend(loc='center left', bbox_to_anchor=(0.02, 0.60))

In [ ]:
# Drawing comparison table
print(f"{'Google':>13} \ {'rating':>10} \ {'reactions per app': >30} \ {'totall apps': >20} \ ")
print('---' * 28)
print(f"{'overall':>13} \ {df_ggl['rating'].mean():>10.2f} \ {df_ggl['rating_count'].sum() / df_ggl['app_name'].shape[0]:>30.2f} \ {df_ggl['app_name'].shape[0]:>20} \ ")
print(f"{'english':>13} \ {ggl_english['rating'].mean():>10.2f} \ {ggl_english['rating_count'].sum() / ggl_english['app_name'].shape[0]:>30.2f} \ {ggl_english['app_name'].shape[0]:>20} \ ")
print(f"{'non-english':>13} \ {ggl_local['rating'].mean():>10.2f} \ {ggl_local['rating_count'].sum() / ggl_local['app_name'].shape[0]:>30.2f} \ {ggl_local['app_name'].shape[0]:>20} \ ")
print()
print(f"{'IOS':>13} \ {'rating':>10} \ {'reactions per app': >30} \ {'totall apps': >20} \ ")
print('---' * 28)
print(f"{'overall':>13} \ {df_ios['rating'].mean():>10.2f} \ {df_ios['rating_count'].sum() / df_ios['app_name'].shape[0]:>30.2f} \ {df_ios['app_name'].shape[0]:>20} \ ")
print(f"{'english':>13} \ {ios_english['rating'].mean():>10.2f} \ {ios_english['rating_count'].sum() / ios_english['app_name'].shape[0]:>30.2f} \ {ios_english['app_name'].shape[0]:>20} \ ")
print(f"{'non-english':>13} \ {ios_local['rating'].mean():>10.2f} \ {ios_local['rating_count'].sum() / ios_local['app_name'].shape[0]:>30.2f} \ {ios_local['app_name'].shape[0]:>20} \ ")

English applications are more popular on both platforms, but on android local apps have more loyal audience. There is also a big possibility that some apps that are named with lating letters are non-english apps and vice versa. As a conclusion here we desire to make english app on android as this type of apps has 2.7 times higher reactions to app ratio.

## Free or paid apps

For this assumption we will not take into account different ways of monetization, but simply check the statistics for paid and free applications.

In [ ]:
# Splitting our dataframes to compare them
ggl_free = df_ggl[df_ggl['price'] == 0]
ggl_paid = df_ggl[df_ggl['price'] != 0]
ios_free = df_ios[df_ios['price'] == 0]
ios_paid = df_ios[df_ios['price'] != 0]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(18, 6))

#Printing KDE for each group
graph = sns.kdeplot(ggl_paid['rating'], color='#FF961F', label='Paid apps', bw=.1, ax=ax1)
graph = sns.kdeplot(ggl_free['rating'], color='#219ebc', label='Free apps', bw=.1, ax=ax1)
graph = sns.kdeplot(df_ggl['rating'], color='#023047', label='All apps', bw=.1, ax=ax1)

#Printing their means
graph.axvline(x = ggl_paid['rating'].mean(), color = '#fb8500', linewidth = 3, alpha = .7)
graph.axvline(x = ggl_free['rating'].mean(), color = '#219ebc', linewidth = 4, alpha = .7)
graph.axvline(x = df_ggl['rating'].mean(), color = '#023047', linewidth = 3, alpha = .7)

#Graphic details
graph.text(x = 2.1, y = 0.9, s = "Play market", fontsize = 20, weight = 'bold', alpha = .75)
graph.text(x = 2.1, y = 0.85, s = 'app rating depending on fee', fontsize = 16, alpha = .85)
graph.tick_params(axis = 'both', which = 'major', labelsize = 16)
graph.axhline(y = 0, color = 'black', linewidth = 4, alpha = .7)
#graph.set_yticklabels(labels = [])
graph.set_xlim(left = 1.9, right = 5)
graph.xaxis.label.set_visible(False)
ax1.legend(loc='center left', bbox_to_anchor=(0.02, 0.60))

#Printing histograms for categorial data
bins = [1.9, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75, 5]

graph = sns.distplot(ios_paid['rating'], norm_hist=True, color='#FF961F',
                     label='Paid apps', bins=bins, ax=ax2, kde=False, hist_kws={"histtype": "bar","alpha": .7})
graph = sns.distplot(ios_free['rating'], norm_hist=True, color='#219ebc',
                     label='Free apps', bins=bins, ax=ax2, kde=False, hist_kws={"histtype": "bar", "alpha": .7})
graph = sns.distplot(df_ios['rating'], norm_hist=True, color='#023047',
                     label='All apps', bins=bins, ax=ax2, kde=False, hist_kws={"histtype": "step", "alpha": 1, 'linewidth': 5})

#Printing their means
graph.axvline(x = ios_paid['rating'].mean(), color = '#FF961F', linewidth = 3, alpha = 1)
graph.axvline(x = ios_free['rating'].mean(), color = '#219ebc', linewidth = 5, alpha = 1)
graph.axvline(x = df_ios['rating'].mean(), color = '#023047', linewidth = 3, alpha = 1)

#Graphic details
graph.text(x = 2.1, y = 0.90, s = "Appstore", fontsize = 20, weight = 'bold', alpha = .75)
graph.text(x = 2.1, y = 0.85, s = 'app rating depending on fee', fontsize = 16, alpha = .85)
graph.tick_params(axis = 'both', which = 'major', labelsize = 16)
graph.axhline(y = 0, color = 'black', linewidth = 4, alpha = .7)
#graph.set_yticklabels(labels = [])
graph.set_xlim(left = 1.9, right = 5)
#graph.set_ylim(bottom = 0, top = 0.00001)
graph.xaxis.label.set_visible(False)
#ax2.legend([],[], frameon=False)
ax2.legend(loc='center left', bbox_to_anchor=(0.02, 0.60))

In [ ]:
# Drawing comparison table 
print(f"{'Google':>13} \ {'rating':>20} \ {'reactions per app': >20} \ {'totall apps': >20} \ ")
print('---' * 28)
print(f"{'overall':>13} \ {df_ggl['rating'].mean():>20.2f} \ {df_ggl['rating_count'].sum() / df_ggl['app_name'].shape[0]:>20.2f} \ {df_ggl['app_name'].shape[0]:>20} \ ")
print(f"{'free':>13} \ {ggl_free['rating'].mean():>20.2f} \ {ggl_free['rating_count'].sum() / ggl_free['app_name'].shape[0]:>20.2f} \ {ggl_free['app_name'].shape[0]:>20} \ ")
print(f"{'paid':>13} \ {ggl_paid['rating'].mean():>20.2f} \ {ggl_paid['rating_count'].sum() / ggl_paid['app_name'].shape[0]:>20.2f} \ {ggl_paid['app_name'].shape[0]:>20} \ ")
print()
print(f"{'IOS':>13} \ {'rating':>20} \ {'reactions per app': >20} \ {'totall apps': >20} \ ")
print('---' * 28)
print(f"{'overall':>13} \ {df_ios['rating'].mean():>20.2f} \ {df_ios['rating_count'].sum() / df_ios['app_name'].shape[0]:>20.2f} \ {df_ios['app_name'].shape[0]:>20} \ ")
print(f"{'free':>13} \ {ios_free['rating'].mean():>20.2f} \ {ios_free['rating_count'].sum() / ios_free['app_name'].shape[0]:>20.2f} \ {ios_free['app_name'].shape[0]:>20} \ ")
print(f"{'paid':>13} \ {ios_paid['rating'].mean():>20.2f} \ {ios_paid['rating_count'].sum() / ios_paid['app_name'].shape[0]:>20.2f} \ {ios_paid['app_name'].shape[0]:>20} \ ")

Conclusion is we would either prefer making paid app for IOS (with more reactions / totall apps ratio) or free app for Play Market, where customers an app at average receives 2 times more ratings which is a synonim of more app installs. If i need to choose one i would pick android option because of the volume and quality of the dataset.

## Content rating comparison

In [ ]:
# Inspect given categories
print('Play Market content categories:')
print(df_ggl['content_rating'].value_counts())
print()
print('Appstore content categories:')
print(df_ios['content_rating'].value_counts())

For the sake of practice and for better comparison let's regroup the content rating values into the new categories:
* 'Unrated' (ggl) and 'Not yet rated' (ios) -> Unrated
* 'Everyone 10+' (ggl) and '4+' (ios) -> 4+
* '10+' (ggl) and '9+' (ios) -> 9+ 
* 'Teen' (ggl) and '12+' (ios) -> 12+ 
* 'Mature 17+' (ggl) and '17+' (ios) -> '17+'
* 'Adults only 18+' (ggl) -> '18+'

In [ ]:
# Let's make function to rename existing content_rating categories
def content(i):
    if i == 'Unrated' or i == 'Not yet rated':
        i = 'Unrated'
    elif i == 'Everyone' or i == '4+':
        i = '4+'
    elif i == 'Everyone 10+' or i == '9+':
        i = '9+'     
    elif i == 'Teen' or i == '12+':
        i = '12+'
    elif i == 'Mature 17+' or i == '17+':
        i = '17+'
    elif i == 'Adults only 18+':
        i = '18+'
    return i

In [ ]:
# Applying content fuction
df_ggl['content_rating'] = df_ggl['content_rating'].map(content)
df_ios['content_rating'] = df_ios['content_rating'].map(content)

In [ ]:
# Splitting our dataframes to compare them
ggl_4 = df_ggl[df_ggl['content_rating'] == '4+']
ggl_10 = df_ggl[df_ggl['content_rating'] == '9+']
ggl_12 = df_ggl[df_ggl['content_rating'] == '12+'] 
ggl_17 = df_ggl[df_ggl['content_rating'] == '17+']
ggl_18 = df_ggl[df_ggl['content_rating'] == '18+']
ggl_unrated = df_ggl[df_ggl['content_rating'] == 'Unrated']

ios_4 = df_ios[df_ios['content_rating'] == '4+']
ios_10 = df_ios[df_ios['content_rating'] == '9+']
ios_12 = df_ios[df_ios['content_rating'] == '12+']
ios_17 = df_ios[df_ios['content_rating'] == '17+']
ios_unrated = df_ios[df_ios['content_rating'] == 'Unrated']

In [ ]:
# Drawing comparison table 
print(f"{'Google':>8} \ {'rating':>20} \ {'reactions per app': >20} \ {'totall apps': >20} \ ")
print('---' * 28)
print(f"{'overall':>8} \ {df_ggl['rating'].mean():>20.2f} \ {df_ggl['rating_count'].sum() / df_ggl['app_name'].shape[0]:>20.2f} \ {df_ggl['app_name'].shape[0]:>20} \ ")
print(f"{'4+':>8} \ {ggl_4['rating'].mean():>20.2f} \ {ggl_4['rating_count'].sum() / ggl_4['app_name'].shape[0]:>20.2f} \ {ggl_4['app_name'].shape[0]:>20} \ ")
print(f"{'10+':>8} \ {ggl_10['rating'].mean():>20.2f} \ {ggl_10['rating_count'].sum() / ggl_10['app_name'].shape[0]:>20.2f} \ {ggl_10['app_name'].shape[0]:>20} \ ")
print(f"{'12+':>8} \ {ggl_12['rating'].mean():>20.2f} \ {ggl_12['rating_count'].sum() / ggl_12['app_name'].shape[0]:>20.2f} \ {ggl_12['app_name'].shape[0]:>20} \ ")
print(f"{'17+':>8} \ {ggl_17['rating'].mean():>20.2f} \ {ggl_17['rating_count'].sum() / ggl_17['app_name'].shape[0]:>20.2f} \ {ggl_17['app_name'].shape[0]:>20} \ ")
print(f"{'18+':>8} \ {ggl_18['rating'].mean():>20.2f} \ {ggl_18['rating_count'].sum() / ggl_18['app_name'].shape[0]:>20.2f} \ {ggl_18['app_name'].shape[0]:>20} \ ")
print(f"{'Unrated':>8} \ {ggl_unrated['rating'].mean():>20.2f} \ {ggl_unrated['rating_count'].sum() / ggl_unrated['app_name'].shape[0]:>20.2f} \ {ggl_unrated['app_name'].shape[0]:>20} \ ")

print()

print(f"{'IOS':>8} \ {'rating':>20} \ {'reactions per app': >20} \ {'totall apps': >20} \ ")
print('---' * 28)
print(f"{'overall':>8} \ {df_ios['rating'].mean():>20.2f} \ {df_ios['rating_count'].sum() / df_ios['app_name'].shape[0]:>20.2f} \ {df_ios['app_name'].shape[0]:>20} \ ")
print(f"{'4+':>8} \ {ios_4['rating'].mean():>20.2f} \ {ios_4['rating_count'].sum() / ios_4['app_name'].shape[0]:>20.2f} \ {ios_4['app_name'].shape[0]:>20} \ ")
print(f"{'10+':>8} \ {ios_10['rating'].mean():>20.2f} \ {ios_10['rating_count'].sum() / ios_10['app_name'].shape[0]:>20.2f} \ {ios_10['app_name'].shape[0]:>20} \ ")
print(f"{'12+':>8} \ {ios_12['rating'].mean():>20.2f} \ {ios_12['rating_count'].sum() / ios_12['app_name'].shape[0]:>20.2f} \ {ios_12['app_name'].shape[0]:>20} \ ")
print(f"{'17+':>8} \ {ios_17['rating'].mean():>20.2f} \ {ios_17['rating_count'].sum() / ios_17['app_name'].shape[0]:>20.2f} \ {ios_17['app_name'].shape[0]:>20} \ ")
print(f"{'Unrated':>8} \ {ios_unrated['rating'].mean():>20.2f} \ {ios_unrated['rating_count'].sum() / ios_unrated['app_name'].shape[0]:>20.2f} \ {ios_unrated['app_name'].shape[0]:>20} \ ")

For such amount of groups pivot table are preferable:

In [ ]:
# Creating pivot table for content rating for Google
table_content_ggl = pd.pivot_table(df_ggl, index=['content_rating'], values = ['rating', 'rating_count', 'app_name'], 
                           aggfunc={'rating':'mean', 'rating_count':'mean', 'app_name':'count'}, margins=True, margins_name='Overall')
table_content_ggl = table_content_ggl[['rating', 'rating_count', 'app_name']].rename(columns={'app_name':'totall_apps', 'rating_count':'reactions per app'})
table_content_ggl

In [ ]:

# Creating pivot table for content rating for IOS
table_content_ios = pd.pivot_table(df_ios, index=['content_rating'], values = ['rating', 'rating_count', 'app_name'], 
                           aggfunc={'rating':'mean', 'rating_count':'mean', 'app_name':'count'}, margins=True, margins_name='Overall')

table_content_ios = table_content_ios[['rating', 'rating_count', 'app_name']].rename(columns={'app_name':'totall_apps', 'rating_count':'reactions per app'})
table_content_ios

In [ ]:
plt.style.use('fivethirtyeight')
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(18, 6))

#Printing KDE for each group
graph = sns.kdeplot(ggl_4['rating'], label='4+', color='#FF961F', bw=.1, ax=ax1)
graph = sns.kdeplot(ggl_10['rating'], label='9+', color='#FFB703', bw=.1, ax=ax1)
graph = sns.kdeplot(ggl_12['rating'], label='12+', color='#d62828', bw=.1, ax=ax1)
graph = sns.kdeplot(ggl_17['rating'], label='17+', color='#219EBC', bw=.1, ax=ax1)
graph = sns.kdeplot(df_ggl['rating'], label='All apps', bw=.1, color='#023047', ax=ax1)

#Printing their means
graph.axvline(x = ggl_4['rating'].mean(), color = '#FF961F', linewidth = 3, alpha = .7)
graph.axvline(x = ggl_17['rating'].mean(), color = '#219ebc', linewidth = 3, alpha = .7)
graph.axvline(x = ggl_10['rating'].mean(), color = '#FFB703', linewidth = 5, alpha = .7)
graph.axvline(x = ggl_12['rating'].mean(), color = '#d62828', linewidth = 3, alpha = .7)
graph.axvline(x = df_ggl['rating'].mean(), color = '#023047', linewidth = 3, alpha = .7)

#Graphic details
graph.text(x = 2.05, y = 0.9, s = "Play market", fontsize = 20, weight = 'bold', alpha = .75)
graph.text(x = 2.05, y = 0.85, s = 'app rating depending on content rating', fontsize = 16, alpha = .85)
graph.tick_params(axis = 'both', which = 'major', labelsize = 16)
graph.axhline(y = 0, color = 'black', linewidth = 4, alpha = .7)
#graph.set_yticklabels(labels = [])
graph.set_xlim(left = 1.9, right = 5)
graph.xaxis.label.set_visible(False)
ax1.legend(loc='center left', bbox_to_anchor=(0.02, 0.59))

#Printing histograms for categorial data
bins = [1.9, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75, 5]
graph = sns.distplot(ios_4['rating'], norm_hist=True, label='4+', color='#FF961F',
                     bins=bins, ax=ax2, kde=False, hist_kws={"histtype": "bar","alpha": .7})
graph = sns.distplot(ios_10['rating'], norm_hist=True, label='9+', color='#FFB703',
                     bins=bins, ax=ax2, kde=False, hist_kws={"histtype": "bar","alpha": .7})
graph = sns.distplot(ios_12['rating'], norm_hist=True, label='12+', color='#d62828',
                     bins=bins, ax=ax2, kde=False, hist_kws={"histtype": "bar","alpha": .7})
graph = sns.distplot(ios_17['rating'], norm_hist=True, label='17+', color='#219EBC',
                     bins=bins, ax=ax2, kde=False, hist_kws={"histtype": "bar","alpha": .7})
graph = sns.distplot(df_ios['rating'], norm_hist=True, label='All apps', color='#023047',
                     bins=bins, ax=ax2, kde=False, hist_kws={"histtype": "step", "alpha": 1, 'linewidth': 5})

#Printing their means
graph.axvline(x = ios_4['rating'].mean(), color = '#FF961F', linewidth = 3, alpha = .7)
graph.axvline(x = ios_10['rating'].mean(), color = '#FFB703', linewidth = 5, alpha = .7)
graph.axvline(x = ios_12['rating'].mean(), color = '#d62828', linewidth = 3, alpha = .7)
graph.axvline(x = ios_17['rating'].mean(), color = '#219ebc', linewidth = 3, alpha = .7)
graph.axvline(x = df_ios['rating'].mean(), color = '#023047', linewidth = 3, alpha = .7)


#Graphic details
graph.text(x = 2.05, y = 0.95, s = "Appstore", fontsize = 20, weight = 'bold', alpha = .75)
graph.text(x = 2.05, y = 0.90, s = 'app rating depending on content rating', fontsize = 16, alpha = .85)
graph.tick_params(axis = 'both', which = 'major', labelsize = 16)
graph.axhline(y = 0, color = 'black', linewidth = 4, alpha = .7)
#graph.set_yticklabels(labels = [])
graph.set_xlim(left = 1.9, right = 5)
#graph.set_ylim(bottom = 0, top = 0.00001)
graph.xaxis.label.set_visible(False)
ax2.legend(loc='center left', bbox_to_anchor=(0.02, 0.59))

As we can see there apps rated 12+ perform better than other content categories on both platforms following by closest category - 9+ with similar average rating but much lower number of reactions on ios. Our choice here is 12+ application for android market with 63k+ ratings per app on average.

## Categories

In [ ]:
# Checking number of categories
print('Number of unique categories in Play Market:',df_ggl['category'].unique().shape[0])
print('Number of unique categories in Play Market:',df_ios['category'].unique().shape[0])

There are totall of 48 categories in Play Market data and 24 in Appstore, so we are going to implement some limitations based on our metrics, to make result more clear. Limitations based on rating and reactions count are refered to mean values of both datasets and also we limit minimum number of apps into category to escape skewed data. Let's assume cutting all categories with rating and rating counts lower that totall mean values and categories with less than 500 apps in them. 


In [ ]:
# Creating pivot table for Play Market app categories
table_category_ggl = pd.pivot_table(df_ggl, index=['category'], values = ['rating', 'rating_count', 'app_name'], 
                           aggfunc={'rating':'mean', 
                                    'rating_count': ['mean', 'median'],
                                    'app_name':'count'})

table_category_ggl.columns = ['totall_apps', 'rating', 'reactions_mean', 'reactions_median']
table_category_ggl = table_category_ggl[(table_category_ggl['reactions_mean'] > 10800) 
                                & (table_category_ggl['rating'] > 4.11) & (table_category_ggl['totall_apps'] > 500)]
table_category_ggl.sort_values(by=['reactions_mean', 'reactions_median'], ascending=False).head()

In [ ]:
# Creating pivot table for Appstore app categories
table_category_ios = pd.pivot_table(df_ios, index=['category'], values = ['rating', 'rating_count', 'app_name'], 
                           aggfunc={'rating':'mean', 
                                    'rating_count': ['mean', 'median'],
                                    'app_name':'count'})

table_category_ios.columns = ['totall_apps', 'rating', 'reactions_mean', 'reactions_median']
table_category_ios = table_category_ios[(table_category_ios['reactions_mean'] > 10800) 
                                & (table_category_ios['rating'] > 4.11) & (table_category_ios['totall_apps'] > 500)]
table_category_ios.sort_values(by=['reactions_mean', 'reactions_median'], ascending=False)

* Games category in Google dataset are split into different categories. One opportunity for us, if we choose to make a game app, we can decide what it would be about.
* We also calculated a median to account for outliers in such categories as Social for Android. We're interested in categories where applications are generally perceived better by the audience, i.e. those where the median is closer to the mean.
* Low median value for Social is related to market giants and make this category less viable.

In Play Market we prepicked 5 categories including 3 games: Casino, Puzzle, Word. And 4 categories in Appstore dataset.

In [ ]:
# Manually pick categories
ggl_social = df_ggl[df_ggl['category'] == 'Social']
ggl_casino = df_ggl[df_ggl['category'] == 'Casino']
ggl_adventure = df_ggl[df_ggl['category'] == 'Adventure']
ggl_puzzle = df_ggl[df_ggl['category'] == 'Puzzle']
ggl_word = df_ggl[df_ggl['category'] == 'Word']

ios_foodrink =df_ios[df_ios['category'] == 'Food & Drink']
ios_photo = df_ios[df_ios['category'] == 'Photo & Video']
ios_finance = df_ios[df_ios['category'] == 'Finance']
ios_reference = df_ios[df_ios['category'] == 'Reference']

In [ ]:
plt.style.use('fivethirtyeight')
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(18, 6))

#Printing KDE for each group
graph = sns.kdeplot(ggl_social['rating'], label='Social', color='#FFB703', bw=.1, ax=ax1)
graph = sns.kdeplot(ggl_casino['rating'], label='Casino', color='#FF961F', bw=.1, ax=ax1)
graph = sns.kdeplot(ggl_adventure['rating'], label='Adventure', color='#d62828', bw=.1, ax=ax1)
graph = sns.kdeplot(ggl_puzzle['rating'], label='Puzzle', color='#219EBC', bw=.1, ax=ax1)
graph = sns.kdeplot(ggl_word['rating'], label='Word', color='#7400b8', bw=.1, ax=ax1)
graph = sns.kdeplot(df_ggl['rating'], label='All apps', bw=.1, color='#023047', ax=ax1)

#Printing their means
graph.axvline(x = ggl_social['rating'].mean(), color = '#FFB703', linewidth = 3, alpha = .7)
graph.axvline(x = ggl_casino['rating'].mean(), color = '#FF961F', linewidth = 3, alpha = .7)
graph.axvline(x = ggl_adventure['rating'].mean(), color = '#d62828', linewidth = 3, alpha = .7)
graph.axvline(x = ggl_puzzle['rating'].mean(), color = '#219EBC', linewidth = 3, alpha = .7)
graph.axvline(x = ggl_word['rating'].mean(), color = '#7400b8', linewidth = 3, alpha = .7)
graph.axvline(x = df_ggl['rating'].mean(), color = '#023047', linewidth = 3, alpha = .7)

#Graphic details
graph.text(x = 2.05, y = 1.3, s = "Play market", fontsize = 20, weight = 'bold', alpha = .75)
graph.text(x = 2.05, y = 1.23, s = 'app rating depending on category', fontsize = 16, alpha = .85)
graph.tick_params(axis = 'both', which = 'major', labelsize = 16)
graph.axhline(y = 0, color = 'black', linewidth = 4, alpha = .7)
#graph.set_yticklabels(labels = [])
graph.set_xlim(left = 1.9, right = 5)
graph.xaxis.label.set_visible(False)
ax1.legend(loc='center left', bbox_to_anchor=(0.02, 0.59))

#Printing histograms for categorial data
bins = [1.9, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75, 5]
graph = sns.distplot(ios_foodrink['rating'], norm_hist=True, label='Food & Drink', color='#7400b8', bins=bins, ax=ax2, kde=False, hist_kws={"histtype": "bar","alpha": .7})
graph = sns.distplot(ios_photo['rating'], norm_hist=True, label='Photo & Video', color='#FFB703', bins=bins, ax=ax2, kde=False, hist_kws={"histtype": "bar","alpha": .7})
graph = sns.distplot(ios_finance['rating'], norm_hist=True, label='Finance', color='#d62828', bins=bins, ax=ax2, kde=False, hist_kws={"histtype": "bar","alpha": .7})
graph = sns.distplot(ios_reference['rating'], norm_hist=True, label='Reference', color='#219EBC', bins=bins, ax=ax2, kde=False, hist_kws={"histtype": "bar","alpha": .7})
graph = sns.distplot(df_ios['rating'], norm_hist=True, label='All apps', color='#023047', bins=bins, ax=ax2, kde=False, hist_kws={"histtype": "step", "alpha": 1, 'linewidth': 5})

#Printing their means
graph.axvline(x = ios_foodrink['rating'].mean(), color = '#7400b8', linewidth = 3, alpha = .7)
graph.axvline(x = ios_photo['rating'].mean(), color = '#FFB703', linewidth = 3, alpha = .7)
graph.axvline(x = ios_finance['rating'].mean(), color = '#d62828', linewidth = 3, alpha = .7)
graph.axvline(x = ios_reference['rating'].mean(), color = '#219ebc', linewidth = 3, alpha = .7)
graph.axvline(x = df_ios['rating'].mean(), color = '#023047', linewidth = 3, alpha = .7)

#Graphic details
graph.text(x = 2.05, y = 1.3, s = "Appstore", fontsize = 20, weight = 'bold', alpha = .75)
graph.text(x = 2.05, y = 1.23, s = 'app rating depending on category', fontsize = 16, alpha = .85)
graph.tick_params(axis = 'both', which = 'major', labelsize = 16)
graph.axhline(y = 0, color = 'black', linewidth = 4, alpha = .7)
#graph.set_yticklabels(labels = [])
graph.set_xlim(left = 1.9, right = 5)
#graph.set_ylim(bottom = 0, top = 0.00001)
graph.xaxis.label.set_visible(False)
ax2.legend(loc='center left', bbox_to_anchor=(0.02, 0.59))

All these categories perform better than average app on the market. Following our previous picks we can't pick Casino here even though it has much better metrics and must be considered for further analysis. Social and Adventure categories have high differences between mean and meadian values and must be checked for outliers.

To make a conclusion our final choice is free puzzle/word game for android with 12+ content rating. And second best option is paid photo/video app for ios.

In [ ]:
def names(df, name):
    '''
    Looks for names in the app_name columns and returns 
    dataframe with all occurences
    '''
    list_ = []
    for i in df['app_name']:
        if name.lower() in i.lower():
            list_.append(i)
    df2 = df[df['app_name'].isin(list_)].sort_values(['rating_count'], ascending=False)
    return df2

## What platform is better

1. "Are the same paid apps on ios have more users?"
2. "Are there any apps that are free on android and paid on ios?"

To answer these questions we are going to merge both datasets on app names as a key.

In [ ]:
# Merging dataframes and checking resulting data volume
result = pd.merge(df_ggl, df_ios, on='app_name', suffixes=('_ggl', '_ios'))
print('Total apps after merge:', result.shape[0])
print('Free apps on both markets:',result[(result['price_ggl'] == 0) & (result['price_ios'] == 0)].shape[0])
print('Paid apps on both markets:',result[(result['price_ggl'] > 0) & (result['price_ios'] > 0)].shape[0])
print('Number of apps that are free in Play Market and paid in Appstore:', result[(result['price_ggl'] == 0) & (result['price_ios'] > 0)].shape[0])
print('Number of apps that are paid in Play Market and free in Appstore:', result[(result['price_ggl'] > 0) & (result['price_ios'] == 0)].shape[0])

We got 8384 apps with perfect matched names. Out of which 13.1% apps are paid just like in Appstore dataset. Appstore has more paid applications Play Market among same apps. That's once again confirms the thesis that it could be better overall to run a paid version of application in the Appstore.

Now let's select the applications that differ in price to answer the second question and those that have the same price to answer the first question.

In [ ]:
question_1 = result[(result['price_ggl'] > 0) & (result['price_ios'] > 0)][['rating_count_ggl', 'rating_count_ios', 'price_ggl', 'price_ios']].mean()
question_1

The same direct paid applications have a higher response in Play Market (13661 vs 5768 reactions per app in average). Average price for apps on both markets is nearly the same, around $4.40.

**TO DO:**
* Check casino with related attributes - there's a chance such apps perform better than most.
* Test Social and Adventure categories on Play Market for outliers and test again.
* Look for attribute combination to start as free app in Play Market then making paid version for Appstore.
* Make further exploration for picked options:
    * Monetization model / average price
    * Find competitors and clarify app topic